In [1]:
import gc
import os
import pygrib
import numpy as np
import xarray as xr

from glob import glob
from functools import partial
from os import makedirs as mkdir
from multiprocessing import get_context
from datetime import datetime, timedelta

os.environ['OMP_NUM_THREAD'] = '1'

In [2]:
nlat, xlat = 30, 50
nlon, xlon = -130, -100

In [3]:
def download_grib(url, subset_str, tmp):
    from subprocess import Popen, call
    import requests
    
    filename = url.split('file=')[1].split('&')[0]
    filename = filename.replace('.co.', '.%s.'%subset_str)
        
    if not os.path.isfile(tmp + filename):
        print('Downloading %s\n'%filename)
        
        # r = requests.get(url, allow_redirects=True)
        # open(tmp + filename, 'wb').write(r.content)
        
        # Use call over Popen (blocking)
        cmd = 'wget -O "%s" "%s"'%(tmp + filename, url)
        call(cmd, shell=True)
    
    return filename

In [4]:
# Set up init to use sys.argv later
init = datetime(2020, 10, 8, 12)
yyyy, mm, dd, hh = init.year, init.month, init.day, init.hour

In [5]:
base = 'https://nomads.ncep.noaa.gov/cgi-bin/filter_blend.pl?'
var = '&var_APCP=on'
region = '&subregion=&leftlon={:.2f}&rightlon={:.2f}&toplat={:.2f}&bottomlat={:.2f}'.format(nlon, xlon, xlat, nlat)
mdir = '&dir=%2Fblend.{:04d}{:02d}{:02d}%2F{:02d}%2Fqmd'.format(yyyy, mm, dd, hh)

url_list = []

# Need to fix the data processing below to allow for sub24 leads
for fhr in np.arange(24, 180+1, 6):
    file = 'file=blend.t{:02d}z.qmd.f{:03d}.co.grib2'.format(hh, fhr)
    url_list.append(base + file + var + region + mdir)

In [6]:
tmpdir = '/scratch/general/lustre/u1070830/nbm/tmp/'; mkdir(tmpdir, exist_ok=True)
datadir = '/scratch/general/lustre/u1070830/nbm/'; mkdir(datadir, exist_ok=True)

In [7]:
download_grib_mp = partial(download_grib, subset_str='WR', tmp=tmpdir)
    
with get_context('fork').Pool(len(url_list)) as p:
    p.map(download_grib_mp, url_list, chunksize=1)
    p.close()
    p.join()

In [8]:
flist = sorted(glob(tmpdir + '*.grib2'))

for i, f in enumerate(flist):
    print(i, f)

0 /scratch/general/lustre/u1070830/nbm/tmp/blend.t12z.qmd.f024.WR.grib2
1 /scratch/general/lustre/u1070830/nbm/tmp/blend.t12z.qmd.f030.WR.grib2
2 /scratch/general/lustre/u1070830/nbm/tmp/blend.t12z.qmd.f036.WR.grib2
3 /scratch/general/lustre/u1070830/nbm/tmp/blend.t12z.qmd.f042.WR.grib2
4 /scratch/general/lustre/u1070830/nbm/tmp/blend.t12z.qmd.f048.WR.grib2
5 /scratch/general/lustre/u1070830/nbm/tmp/blend.t12z.qmd.f054.WR.grib2
6 /scratch/general/lustre/u1070830/nbm/tmp/blend.t12z.qmd.f060.WR.grib2
7 /scratch/general/lustre/u1070830/nbm/tmp/blend.t12z.qmd.f066.WR.grib2
8 /scratch/general/lustre/u1070830/nbm/tmp/blend.t12z.qmd.f072.WR.grib2
9 /scratch/general/lustre/u1070830/nbm/tmp/blend.t12z.qmd.f078.WR.grib2
10 /scratch/general/lustre/u1070830/nbm/tmp/blend.t12z.qmd.f084.WR.grib2
11 /scratch/general/lustre/u1070830/nbm/tmp/blend.t12z.qmd.f090.WR.grib2
12 /scratch/general/lustre/u1070830/nbm/tmp/blend.t12z.qmd.f096.WR.grib2
13 /scratch/general/lustre/u1070830/nbm/tmp/blend.t12z.qmd.f1

In [9]:
f = flist[0]
print(f)

grb = pygrib.open(f)
msgs = grb.read()

msgs

/scratch/general/lustre/u1070830/nbm/tmp/blend.t12z.qmd.f024.WR.grib2


[1:Total Precipitation:kg m**-2 (accum):lambert:surface:level 0:fcst time 18-24 hrs (accum):from 202010081200:Probability of event above upper limit (> 0.254),
 2:Total Precipitation:kg m**-2 (accum):lambert:surface:level 0:fcst time 18-24 hrs (accum):from 202010081200,
 3:Total Precipitation:kg m**-2 (accum):lambert:surface:level 0:fcst time 18-24 hrs (accum):from 202010081200,
 4:Total Precipitation:kg m**-2 (accum):lambert:surface:level 0:fcst time 18-24 hrs (accum):from 202010081200,
 5:Total Precipitation:kg m**-2 (accum):lambert:surface:level 0:fcst time 18-24 hrs (accum):from 202010081200,
 6:Total Precipitation:kg m**-2 (accum):lambert:surface:level 0:fcst time 18-24 hrs (accum):from 202010081200,
 7:Total Precipitation:kg m**-2 (accum):lambert:surface:level 0:fcst time 18-24 hrs (accum):from 202010081200,
 8:Total Precipitation:kg m**-2 (accum):lambert:surface:level 0:fcst time 18-24 hrs (accum):from 202010081200,
 9:Total Precipitation:kg m**-2 (accum):lambert:surface:level 0

In [ ]:

# for msg in msgs:

#     interval = msg['stepRange'].split('-')
#     interval = int(interval[1]) - int(interval[0])

#     if 'Probability of event' in str(msg):    
#             # Probability of event above upper limit (> 0.254) NOT inclusive
#             threshold = round(msg['upperLimit']/25.4, 2)
#             probability.append([msg.values])
#             probability_labels.append([interval, threshold])

#     elif 'percentileValue' in msg.keys():                
#         percentile.append([msg.values])
#         percentile_labels.append([interval, msg['percentileValue']])

#     else:
#         if got_deterministic[interval] == False:
#             deterministic_labels.append(interval)
#             deterministic.append(msg.values)
#             got_deterministic[interval] = True
#         else:
#             pass

# grb.close()
# gc.collect()

# deterministic_labels = np.array(deterministic_labels)
# deterministic_labels = deterministic_labels[np.argsort(deterministic_labels)]
# deterministic = np.array(deterministic)[np.argsort(deterministic_labels)]

# probability = np.array(probability, dtype=object).reshape(-1, lats.shape[0], lats.shape[1])
# probability_labels = np.array(probability_labels)

# percentile = np.array(percentile, dtype=object).reshape(-1, 99, lats.shape[0], lats.shape[1])
# percentile_labels = np.array(percentile_labels)

# deterministic = xr.DataArray(deterministic.astype(np.float32), name='pop',
#                 dims=('interval', 'y', 'x'),
#                 coords={'interval':('interval', deterministic_labels),
#                         'lats':(('y', 'x'), lats), 'lons':(('y', 'x'), lons)})

# pop = xr.DataArray(probability[:3].astype(np.float32), name='pop',
#                 dims=('interval', 'y', 'x'),
#                 coords={'interval':('interval', probability_labels[:3, 0]),
#                         'lats':(('y', 'x'), lats), 'lons':(('y', 'x'), lons)})

# probability = xr.DataArray([probability[2:].astype(np.float32)], name='probability',
#                 dims=('interval', 'threshold', 'y', 'x'),
#                 coords={'interval':('interval', [24]), 'threshold':('threshold', probability_labels[2:,1]),
#                         'lats':(('y', 'x'), lats), 'lons':(('y', 'x'), lons)})

# percentile = xr.DataArray(percentile.astype(np.float32), name='percentile',
#                 dims=('interval', 'percentile', 'y', 'x'),
#                 coords={'interval':('interval', np.unique(percentile_labels[:, 0])), 'percentile':('percentile', range(1, 100)),
#                         'lats':(('y', 'x'), lats), 'lons':(('y', 'x'), lons)})

# ds = xr.Dataset()

# # ds['fhr'] = fhr
# ds['time'] = valid
# ds.attrs['InitTime'] = str(init)

# ds['qpf'] = deterministic
# ds['pop'] = pop
# ds['probx'] = probability
# ds['pqpf'] = percentile